In [74]:
import numpy as np
import numpy.typing as npt
import math
from numba import config, njit, prange
import pandas as pd
from numpy_atomic import atomic_add
from typing import Tuple, Callable
from utils import fft_3D_real
from morton import positions_to_keys
import pnumpy

In [118]:
@njit(["f4[:,:,::1](f4[:,:,::1], f4)"], fastmath=True, cache=True, parallel=True)
def operator2(x: npt.NDArray[np.float32], h: np.float32) -> npt.NDArray[np.float32]:
    """Laplacian operator

    Parameters
    ----------
    x : npt.NDArray[np.float32]
        Potential [N_cells_1d, N_cells_1d, N_cells_1d]
    h : np.float32
        Grid size

    Returns
    -------
    npt.NDArray[np.float32]
        Laplacian(x) [N_cells_1d, N_cells_1d, N_cells_1d]
    """
    invh2 = np.float32(h ** (-2))
    six = np.float32(6)
    # Initialise mesh
    result = np.empty_like(x)
    # Computation
    for i in prange(-1, x.shape[0] - 1):
        im1 = i - 1
        ip1 = i + 1
        for j in prange(-1, x.shape[1] - 1):
            jm1 = j - 1
            jp1 = j + 1
            for k in prange(-1, x.shape[2] - 1):
                km1 = k - 1
                kp1 = k + 1
                # Put in array
                result[i, j, k] = (
                    x[im1, j, k]
                    + x[i, jm1, k]
                    + x[i, j, km1]
                    - six * x[i, j, k]
                    + x[i, j, kp1]
                    + x[i, jp1, k]
                    + x[ip1, j, k]
                ) * invh2
    return result

In [119]:
@njit(["f4[:,:,::1](f4[:,:,::1], f4)"], fastmath=True, cache=True, parallel=True)
def operator(x: npt.NDArray[np.float32], h: np.float32) -> npt.NDArray[np.float32]:
    """Laplacian operator

    Parameters
    ----------
    x : npt.NDArray[np.float32]
        Potential [N_cells_1d, N_cells_1d, N_cells_1d]
    h : np.float32
        Grid size

    Returns
    -------
    npt.NDArray[np.float32]
        Laplacian(x) [N_cells_1d, N_cells_1d, N_cells_1d]
    """
    invh2 = np.float32(h ** (-2))
    six = np.float32(6)
    ncells_1d = x.shape[0]
    # Initialise mesh
    result = np.empty_like(x)
    # Computation
    for i in prange(-1, ncells_1d - 1):
        im1 = i - 1
        ip1 = i + 1
        for j in prange(-1, ncells_1d - 1):
            jm1 = j - 1
            jp1 = j + 1
            for k in prange(-1, ncells_1d - 1):
                km1 = k - 1
                kp1 = k + 1
                # Put in array
                result[i, j, k] = (
                    x[im1, j, k]
                    + x[i, jm1, k]
                    + x[i, j, km1]
                    - six * x[i, j, k]
                    + x[i, j, kp1]
                    + x[i, jp1, k]
                    + x[ip1, j, k]
                ) * invh2
    return result

In [107]:
n = 1024
h = 1./n
grid = np.random.rand(n,n,n).astype(np.float32)
#grid = np.random.rand(3, n,n,n).astype(np.float32)
#gridT = np.random.rand(n,n,n, 3).astype(np.float32)
#df = pd.read_parquet('/home/mabreton/boxlen500_n256_lcdmw7v2_00000/3LPT/output_00000/particles.parquet')
#pos = np.empty((3, n**3), dtype=np.float32)
#pos[0] = df["x"]
#pos[1] = df["y"]
#pos[2] = df["z"]
#posT = np.ascontiguousarray(np.transpose(pos))
df = 0
#pos = np.random.rand(n**3, 3).astype(np.float32)
#vel = np.random.rand(n**3, 3).astype(np.float32)

In [69]:
pos0 = pos.copy()
vel0 = vel.copy()

In [70]:
index = positions_to_keys(pos0)

In [66]:
np.allclose(operator(grid, h), operator2(grid, h))

True

In [120]:
%timeit operator2(grid, h)

1.51 s ± 59.7 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [121]:
%timeit operator(grid, h)

1.47 s ± 3.03 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [73]:
%time np.argsort(index, kind='heapsort')

NameError: name 'index' is not defined

In [ ]:
%time np.argsort(index, kind='stable')

In [ ]:
%time pnumpy.argsort(index, kind='quicksort')

In [ ]:
%time pnumpy.argsort(index, kind='mergesort')

In [ ]:
%time pnumpy.argsort(index, kind='heapsort')

In [ ]:
%time pnumpy.argsort(index, kind='stable')

In [19]:
np.allclose(np.argsort(index), parallel_sort.sort(index))

NameError: name 'parallel_sort' is not defined

In [21]:
%timeit np.argsort(index)

3.64 s ± 89.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [30]:
%timeit sorting(index)

/home/mabreton/anaconda3/envs/nbody_env3.11/lib/python3.11/site-packages/numba/core/typed_passes.py:334: NumbaPerformanceWarning: 
The keyword argument 'parallel=True' was specified but no transformation for parallel execution was possible.

To find out why, try turning on parallel diagnostics, see https://numba.readthedocs.io/en/stable/user/parallel.html#diagnostics for help.

File "../../../tmp/ipykernel_9659/4236890148.py", line 1:
<source missing, REPL/exec in use?>

  warnings.warn(errors.NumbaPerformanceWarning(msg,


1.58 s ± 37.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [9]:
reorder_particles(pos, vel)

Function -----------------------reorder_particles took 6.305802342999 seconds----------


In [10]:
pos

array([[0.00266004, 0.0036746 , 0.00219189],
       [0.00367713, 0.00348665, 0.00762756],
       [0.00413912, 0.00146136, 0.00557285],
       ...,
       [0.9993176 , 0.99715513, 0.99266255],
       [0.99626535, 0.99612325, 0.99907947],
       [0.9999851 , 0.99999857, 0.9976268 ]], dtype=float32)

In [60]:
grid_k = fft_3D_real(grid, 4)

In [61]:
k, pk, nm = fourier_grid_to_Pk(grid_k,2)

In [62]:
k2, pk2, nm2 = fourier_grid_to_Pk_sequential(grid_k,2)

In [63]:
k3, pk3, nm3 = fourier_grid_to_Pk_parallel(grid_k, 2)

In [66]:
%timeit fourier_grid_to_Pk(grid_k,2)

232 ms ± 10.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [67]:
%timeit fourier_grid_to_Pk_sequential(grid_k,2)

163 ms ± 370 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [71]:
%timeit fourier_grid_to_Pk_parallel(grid_k, 2)

161 ms ± 2.78 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [64]:
np.allclose(k, k2), np.allclose(pk, pk2), np.allclose(nm, nm2)

(True, True, True)

In [65]:
np.allclose(k, k3), np.allclose(pk, pk3), np.allclose(nm, nm3)

(False, True, True)

In [ ]:
%timeit TSC(pos,n)

In [ ]:
%timeit TSC2(posT,n)

In [ ]:
%timeit invTSC_vec2(gridT, posT)

In [ ]:
%timeit invTSC_vec(grid, pos)

In [ ]:
%timeit derivative5(griid)

In [ ]:
%timeit derivative52(griid)

In [ ]:
# @utils.time_me
@njit(["void(f4[:,:,::1], f4[:,:,::1], f4)"], fastmath=True, cache=True, parallel=False)
def gauss_seidel(
    x: npt.NDArray[np.float32],
    b: npt.NDArray[np.float32],
    h: np.float32,
) -> None:
    """Gauss-Seidel iteration using red-black ordering without over-relaxation \\
    Smooths x in Laplacian(x) = b

    Parameters
    ----------
    x : npt.NDArray[np.float32]
        Potential [N_cells_1d, N_cells_1d, N_cells_1d]
    b : npt.NDArray[np.float32]
        Right-hand side of Poisson equation [N_cells_1d, N_cells_1d, N_cells_1d]
    h : np.float32
        Grid size
    """
    # WARNING: If I replace the arguments in prange by some constant values (for example, doing imax = int(0.5*x.shape[0]), then prange(imax)...),
    #          then LLVM tries to fuse the red and black loops! And we really don't want that...
    h2 = np.float32(h**2)
    invsix = np.float32(1.0 / 6)
    half_ncells_1d = x.shape[0] >> 1

    # Computation Red
    for i in prange(x.shape[0] >> 1):
        ii = 2 * i
        iim2 = ii - 2
        iim1 = ii - 1
        iip1 = ii + 1
        for j in prange(half_ncells_1d):
            jj = 2 * j
            jjm2 = jj - 2
            jjm1 = jj - 1
            jjp1 = jj + 1
            for k in prange(half_ncells_1d):
                kk = 2 * k
                kkm2 = kk - 2
                kkm1 = kk - 1
                kkp1 = kk + 1
                #
                x001 = x[iim1, jjm1, kk]
                x010 = x[iim1, jj, kkm1]
                x100 = x[ii, jjm1, kkm1]
                x111 = x[ii, jj, kk]
                # Put in array
                x[iim1, jjm1, kkm1] = (
                    +x001
                    + x010
                    + x100
                    + x[iim2, jjm1, kkm1]
                    + x[iim1, jjm2, kkm1]
                    + x[iim1, jjm1, kkm2]
                    - h2 * b[iim1, jjm1, kkm1]
                ) * invsix
                # Put in array
                x[iim1, jj, kk] = (
                    x[iim2, jj, kk]
                    + x001
                    + x010
                    + x111
                    - h2 * b[iim1, jj, kk]
                    + x[iim1, jj, kkp1]
                    + x[iim1, jjp1, kk]
                ) * invsix
                # Put in array
                x[ii, jjm1, kk] = (
                    x001
                    + x100
                    + x111
                    + x[ii, jjm2, kk]
                    - h2 * b[ii, jjm1, kk]
                    + x[ii, jjm1, kkp1]
                    + x[iip1, jjm1, kk]
                ) * invsix
                # Put in array
                x[ii, jj, kkm1] = (
                    x010
                    + x100
                    + x111
                    + x[ii, jj, kkm2]
                    - h2 * b[ii, jj, kkm1]
                    + x[ii, jjp1, kkm1]
                    + x[iip1, jj, kkm1]
                ) * invsix
    # Computation Black
    for i in prange(half_ncells_1d):
        ii = 2 * i
        iim2 = ii - 2
        iim1 = ii - 1
        iip1 = ii + 1
        for j in prange(half_ncells_1d):
            jj = 2 * j
            jjm2 = jj - 2
            jjm1 = jj - 1
            jjp1 = jj + 1
            for k in prange(half_ncells_1d):
                kk = 2 * k
                kkm2 = kk - 2
                kkm1 = kk - 1
                kkp1 = kk + 1
                #
                x000 = x[iim1, jjm1, kkm1]
                x011 = x[iim1, jj, kk]
                x101 = x[ii, jjm1, kk]
                x110 = x[ii, jj, kkm1]
                # Put in array
                x[iim1, jjm1, kk] = (
                    +x000
                    + x011
                    + x101
                    + x[iim2, jjm1, kk]
                    + x[iim1, jjm2, kk]
                    - h2 * b[iim1, jjm1, kk]
                    + x[iim1, jjm1, kkp1]
                ) * invsix
                # Put in array
                x[iim1, jj, kkm1] = (
                    +x000
                    + x011
                    + x110
                    + x[iim2, jj, kkm1]
                    + x[iim1, jj, kkm2]
                    - h2 * b[iim1, jj, kkm1]
                    + x[iim1, jjp1, kkm1]
                ) * invsix
                # Put in array
                x[ii, jjm1, kkm1] = (
                    x000
                    + x101
                    + x110
                    + +x[ii, jjm2, kkm1]
                    + x[ii, jjm1, kkm2]
                    - h2 * b[ii, jjm1, kkm1]
                    + x[iip1, jjm1, kkm1]
                ) * invsix
                # Put in array
                x[ii, jj, kk] = (
                    x011
                    + x101
                    + x110
                    - h2 * b[ii, jj, kk]
                    + x[ii, jj, kkp1]
                    + x[ii, jjp1, kk]
                    + x[iip1, jj, kk]
                ) * invsix



In [ ]:
# @utils.time_me
@njit(["void(f4[:,:,::1], f4[:,:,::1], f4, i4)"], fastmath=True, cache=True, parallel=False)
def gauss_seidel_loop(
    x: npt.NDArray[np.float32],
    b: npt.NDArray[np.float32],
    h: np.float32,
    niter: np.int32
) -> None:
    """Gauss-Seidel iteration using red-black ordering without over-relaxation \\
    Smooths x in Laplacian(x) = b

    Parameters
    ----------
    x : npt.NDArray[np.float32]
        Potential [N_cells_1d, N_cells_1d, N_cells_1d]
    b : npt.NDArray[np.float32]
        Right-hand side of Poisson equation [N_cells_1d, N_cells_1d, N_cells_1d]
    h : np.float32
        Grid size
    """
    # WARNING: If I replace the arguments in prange by some constant values (for example, doing imax = int(0.5*x.shape[0]), then prange(imax)...),
    #          then LLVM tries to fuse the red and black loops! And we really don't want that...
    h2 = np.float32(h**2)
    invsix = np.float32(1.0 / 6)
    half_ncells_1d = x.shape[0] >> 1

    for _ in range(niter):
        # Computation Red
        for i in prange(x.shape[0] >> 1):
            ii = 2 * i
            iim2 = ii - 2
            iim1 = ii - 1
            iip1 = ii + 1
            for j in prange(half_ncells_1d):
                jj = 2 * j
                jjm2 = jj - 2
                jjm1 = jj - 1
                jjp1 = jj + 1
                for k in prange(half_ncells_1d):
                    kk = 2 * k
                    kkm2 = kk - 2
                    kkm1 = kk - 1
                    kkp1 = kk + 1
                    #
                    x001 = x[iim1, jjm1, kk]
                    x010 = x[iim1, jj, kkm1]
                    x100 = x[ii, jjm1, kkm1]
                    x111 = x[ii, jj, kk]
                    # Put in array
                    x[iim1, jjm1, kkm1] = (
                        +x001
                        + x010
                        + x100
                        + x[iim2, jjm1, kkm1]
                        + x[iim1, jjm2, kkm1]
                        + x[iim1, jjm1, kkm2]
                        - h2 * b[iim1, jjm1, kkm1]
                    ) * invsix
                    # Put in array
                    x[iim1, jj, kk] = (
                        x[iim2, jj, kk]
                        + x001
                        + x010
                        + x111
                        - h2 * b[iim1, jj, kk]
                        + x[iim1, jj, kkp1]
                        + x[iim1, jjp1, kk]
                    ) * invsix
                    # Put in array
                    x[ii, jjm1, kk] = (
                        x001
                        + x100
                        + x111
                        + x[ii, jjm2, kk]
                        - h2 * b[ii, jjm1, kk]
                        + x[ii, jjm1, kkp1]
                        + x[iip1, jjm1, kk]
                    ) * invsix
                    # Put in array
                    x[ii, jj, kkm1] = (
                        x010
                        + x100
                        + x111
                        + x[ii, jj, kkm2]
                        - h2 * b[ii, jj, kkm1]
                        + x[ii, jjp1, kkm1]
                        + x[iip1, jj, kkm1]
                    ) * invsix
        # Computation Black
        for i in prange(half_ncells_1d):
            ii = 2 * i
            iim2 = ii - 2
            iim1 = ii - 1
            iip1 = ii + 1
            for j in prange(half_ncells_1d):
                jj = 2 * j
                jjm2 = jj - 2
                jjm1 = jj - 1
                jjp1 = jj + 1
                for k in prange(half_ncells_1d):
                    kk = 2 * k
                    kkm2 = kk - 2
                    kkm1 = kk - 1
                    kkp1 = kk + 1
                    #
                    x000 = x[iim1, jjm1, kkm1]
                    x011 = x[iim1, jj, kk]
                    x101 = x[ii, jjm1, kk]
                    x110 = x[ii, jj, kkm1]
                    # Put in array
                    x[iim1, jjm1, kk] = (
                        +x000
                        + x011
                        + x101
                        + x[iim2, jjm1, kk]
                        + x[iim1, jjm2, kk]
                        - h2 * b[iim1, jjm1, kk]
                        + x[iim1, jjm1, kkp1]
                    ) * invsix
                    # Put in array
                    x[iim1, jj, kkm1] = (
                        +x000
                        + x011
                        + x110
                        + x[iim2, jj, kkm1]
                        + x[iim1, jj, kkm2]
                        - h2 * b[iim1, jj, kkm1]
                        + x[iim1, jjp1, kkm1]
                    ) * invsix
                    # Put in array
                    x[ii, jjm1, kkm1] = (
                        x000
                        + x101
                        + x110
                        + +x[ii, jjm2, kkm1]
                        + x[ii, jjm1, kkm2]
                        - h2 * b[ii, jjm1, kkm1]
                        + x[iip1, jjm1, kkm1]
                    ) * invsix
                    # Put in array
                    x[ii, jj, kk] = (
                        x011
                        + x101
                        + x110
                        - h2 * b[ii, jj, kk]
                        + x[ii, jj, kkp1]
                        + x[ii, jjp1, kk]
                        + x[iip1, jj, kk]
                    ) * invsix



In [ ]:
n = 128
ncells = n**3
h = np.float32(1./n)

In [ ]:
a = np.random.rand(n,n,n).astype(np.float32)
b = np.random.rand(n,n,n).astype(np.float32)
a1 = a.copy()

In [ ]:
gauss_seidel(a,b, h)

In [ ]:
gauss_seidel_loop(a1,b, h, 1)

In [ ]:
np.allclose(a, a1)

In [ ]:
laplacian_block(a,h)[10,10:12, 10:12]

In [ ]:
%timeit gauss_seidel(a,b,h), gauss_seidel(a,b,h)

In [ ]:
%timeit gauss_seidel_loop(a1,b,h,2)